<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Train machine learning model</font></strong></p>

WALOUS_OCS_FUSION - Copyright (C) <2020> <Service Public de Wallonie (SWP), Belgique,
					          		Institut Scientifique de Service Public (ISSeP), Belgique,
									Université catholique de Louvain (UCLouvain), Belgique,
									Université Libre de Bruxelles (ULB), Belgique>
						 							
	
List of the contributors to the development of WALOUS_OCS_FUSION: see LICENSE file.
Description and complete License: see LICENSE file.
	
This program (WALOUS_OCS_FUSION) is free software: 
you can redistribute it and/or modify it under the terms of the 
GNU General Public License as published by the Free Software 
Foundation, either version 3 of the License, or (at your option) 
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not, 
see <http://www.gnu.org/licenses/>.

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile

** Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath('/export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [31]:
run /export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/config.py

In [32]:
print config_parameters

{'outputfolder_classifRF_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH/classif_tiles', 'outputfolder_classifRF': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH', 'outputfolder_Logfile': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Log_file', 'permanent_mapset': 'PERMANENT', 'GRASSBIN': '/usr/bin/grass76', 'locationepsg': '31370', 'outputfolder': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH', 'list_tiles': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/list_tiles', 'pixel_classes_list': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/pixel_classes_list', 'outputfolder_classifRF_csv': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH/classif_csv', 'gisdb': '/export/miro/cbassine/GRASSDATA', 'location': 'WALOUS_31370', 'outputfolder_classfeatures': '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features', 'PYTHONLIB': '/u

## Special functions

In [6]:
# Import function that check and create folder
from mkdir import check_create_dir
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [7]:
def CheckNumColCsv(fileList):
    import sys,os,csv
    error = False
    if len(fileList)<=1:
        sys.exit("This function require at least two .csv files.")
    # first file header
    filename = fileList[0]
    tile_num = os.path.splitext(os.path.basename(filename))[0].split("_")[-1]
    f=open(filename)
    reader = csv.reader(f)
    count_firstfile = len(reader.next())
    print "Expected number of columns to be found in all files is %s (correspond to tile '%s')"%(count_firstfile,tile_num)
    # now the rest of files
    for filename in fileList[1:]:
        tile_num = os.path.splitext(os.path.basename(filename))[0].split("_")[-1]
        f=open(filename)
        reader = csv.reader(f)
        count_column = len(reader.next())
        if count_column != count_firstfile:
            error = True
            print "ERROR: Tile '%s' has %s columns"%(tile_num,count_column)
    if not error:
        print "All the files have the same number of columns"
    else: 
        sys.exit()

In [8]:
def StackCsv(fileList,outfile,overwrite=False):
    import sys,os,csv
    if len(fileList)<=1:
        sys.exit("This function require at least two .csv files to be stacked together.")
    if os.path.isfile(outfile) and overwrite==False:
        print "File '"+str(outfile)+"' aleady exists and overwrite option is not enabled."
    else:
        if overwrite==True:
            if os.path.exists(outfile):
                os.remove(outfile)
        stacked_csv=open(outfile,"a")  #Open in mode 'append'
        # first file including header:
        for line in open(fileList[0]):
            if len(line.split("\n")[0])>0:
                stacked_csv.write(line)
        # now the rest without header:
        for filename in fileList[1:]:
            f=open(filename)
            header = f.next() # skip the header
            for line in f:
                if len(line.split("\n")[0])>0:
                    stacked_csv.write(line)
            f.close() # not really needed
        stacked_csv.close()
        print str(len(fileList))+" individual .csv files will be stacked each other."

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [9]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_classfeatures'])
check_create_dir(config_parameters['outputfolder_Logfile'])
check_create_dir(config_parameters['outputfolder_classifRF'])
check_create_dir(config_parameters['outputfolder_classifRF_csv'])
check_create_dir(config_parameters['outputfolder_classifRF_tiles'])

The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features' already exists
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Log_file' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH/classif_csv' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classif_RF_TEST_noMNH/classif_tiles' has been created


# Collect training data

## Get list of segment ID

In [25]:
# Read csv
csvfile = os.path.join(config_parameters['outputfolder_training_sample'],"training_points_with_segid.csv")
fin = open(csvfile)
reader = csv.reader(fin)
# Get list of seg_ID
list_segid = []
label_dic = {}
reader.next()
for row in reader:
    tilegid = row[-1] # row[-1] for 'tilegid'
    segid = row[-2] # row[-2] for 'seg_id'
    label = row[1]  # row[-3] for label 'label_2018' #ortho2018
    print tilegid
    print segid
    print label
    if tilegid != "" and segid != "":
        twokeyid = "%s_%s"%(segid,tilegid)
        list_segid.append(twokeyid)   
        label_dic[twokeyid] = label
initial_count = len(list_segid)
# Keep only value that appear once
count = [list_segid.count(a) for a in list_segid]  # Get list of count for each value
list_segid = [list_segid[i] for i,a in enumerate(count) if a == 1]   # Keep only where count = 1
list_segid = [(x.split("_")[0],x.split("_")[1], label_dic["%s_%s"%(x.split("_")[0],x.split("_")[1])]) for x in list_segid]
unique_count = len(list_segid)

# Create dictionnary with unique segments+tile ID
training_dic = {}
segid_list, tileid_list, label_list = zip(*list_segid)
for segid, tileid, label in zip(segid_list, tileid_list, label_list):
    if tileid not in training_dic:
        training_dic[tileid] = {}
    training_dic[tileid][segid] = label
    if "" in training_dic[tileid]:
        del training_dic[tileid][""]
if "" in training_dic:
    del training_dic[""]
print training_dic    
# Print
print "%s segments out of the %s initially available will be used for training."%(unique_count,initial_count)

4692
11266
2
1253
15168
41
630
38021
5
2771
9668
41
2544
106963
5
4692
11424
5
2542
84409
5
257
2666
41
5847
51939
5
2396
74040
5
414
40869
41
4149
151501
41
5648
123729
11
5351
38885
41
450
2586
42
5846
6112
5
5648
65574
5
2484
383
42
2023
284567
11
4755
12855
42
257
3593
41
254
1470
41
3098
11362
42
518
68260
41
426
9025
5
5758
23281
42
4099
116693
41
518
72000
42
1734
203326
12
285
5385
41
631
10890
41
1197
25143
41
305
1625
41
281
6626
42
2399
7378
42
2516
76969
41
5351
23389
41
5758
91970
11
3292
5095
42
3309
135928
5
4134
134751
41
425
28861
41
1208
149972
12
4149
209413
41
4808
21790
42
1896
77436
5
2023
201379
11
462
50021
41
5845
9470
42
3384
87283
12
5054
106707
5
1183
125012
11
4065
19119
41
1793
256169
11
2023
18940
12
2496
27546
5
3872
22231
2
630
16983
5
2147
45870
5
4099
107211
5
4787
2554
42
2397
52852
11
4108
45590
41
1206
10305
12
1840
182196
11
1216
28354
11
254
6471
5
1876
115124
11
1792
105165
11
3826
356
5
4692
159545
11
2821
65950
5
1897
6675
42
3629
126386
11
36

4787
29174
5
5846
8103
5
5050
41010
6
2020
157803
6
2488
31860
11
5796
30197
6
5789
107121
5
2480
83233
5
2546
3953
41
1897
126235
11
5101
53296
5
2957
51649
42
2484
34244
5
3839
60766
41
5066
11998
42
2517
18562
42
2817
43935
5
5788
21962
11
2039
15167
3
6164
602
3
5934
40879
2
2044
13133
3
2483
19617
5
3723
62897
42
1157
165151
42
6164
18748
42
5763
9502
5
2486
33590
42
1848
51106
42
3385
8215
3
2769
37531
42
2197
20310
41
2665
20
5
2039
76664
3
5820
11144
6
2518
13249
5
437
54905
2
3888
54712
42
2024
23520
6
6297
31326
6
437
53441
6
3871
34736
42
5830
35047
6
5050
74897
6
426
14279
6
2017
212706
5
2815
40443
42
5796
99375
12
5294
29590
6
3871
81180
6
1842
268070
6
2022
215315
11
5820
152693
6
5762
236147
6
5054
75820
6
5789
8039
6
413
41479
6
2022
217486
6
5663
77070
5
4748
37787
5
2517
25129
11
2039
52953
11
426
9303
6
2022
162304
5
1208
221907
42
5817
187991
5
3825
35967
6
1793
3899
3
2017
229240
11
305
6346
6
3830
21381
42
2045
66860
11
3629
4296
11
1791
178022
11
1842
304171
11


5398
77769
3
6164
32173
3
3309
1345
3
3310
11079
3
5787
864
3
5797
264835
2
1158
27820
11
1897
35896
12
5762
94789
11
2016
142346
11
1839
1253
5
1791
139354
5
1789
8932
3
5788
70874
5
3723
28570
6
3408
27995
3
2017
247318
11
1840
21935
42
1205
77290
5
3409
66799
3
2041
63605
5
2043
12602
3
3410
26735
3
1876
196559
5
5820
151332
2
5396
59044
3
1842
218930
11
1848
91871
5
5763
178217
42
1158
93924
5
5762
212429
42
6217
8731
2
1896
5286
3
2042
34420
2
2016
83595
11
1790
85517
5
3385
5668
3
2020
180527
12
3721
24455
3
185
581
42
545
38410
42
277
8179
42
288
10336
42
4157
228439
3
1180
34334
42
462
26546
42
4148
1579
41
656
54764
5
498
2155
5
281
27952
5
1059
16962
3
1128
80830
5
518
67996
42
279
32809
5
287
21773
41
305
95888
41
413
8946
11
545
25017
5
5359
3928
41
666
62873
12
498
1446
5
656
13302
11
1068
18591
5
5563
66080
5
680
57723
3
1178
121842
5
305
31108
11
5320
37502
41
305
2489
41
5337
11245
42
1178
24946
3
4028
59215
5
247
38266
41
461
30802
6
5491
31887
41
1253
45388
41
288
398

42
2335
2874
41
2520
115523
41
2334
38339
41
3288
23856
42
2494
30452
42
4787
1906
3
5921
61093
5
5035
4685
3
5843
11714
42
4813
100427
5
3939
61540
42
3826
30470
42
6351
7979
3
2496
33629
5
6333
86764
3
5851
26153
42
3830
13113
3
5846
38177
42
5054
97987
5
2773
12
11
6194
41396
41
2520
85402
42
2521
32251
11
3825
7203
42
4816
1589
41
3054
10005
5
5294
19418
42
2664
7494
42
3288
82609
5
2487
44547
42
2522
56007
5
6357
28998
5
4816
71682
41
2496
18927
42
4814
93882
11
3838
6913
5
5464
6502
41
2219
27731
42
2665
15472
42
3838
3486
5
2147
43754
5
4691
120518
42
3048
34391
41
2664
12958
42
3629
78834
12
4692
113042
11
2817
48671
42
2487
30516
41
4787
22587
5
2770
31320
42
4811
65994
42
2214
37269
41
4781
48291
5
2153
2204
5
2212
12235
42
4809
3585
42
6194
52236
41
2398
57409
5
6325
73105
11
1183
68550
5
2482
46240
2
4064
47715
42
4157
224701
5
5845
54468
5
260
15057
41
4065
8116
41
2519
34509
42
5648
9566
5
2772
2771
42
2820
33948
42
2482
1416
5
4146
6029
5
817
42735
12
632
31293
42
2496
2

## Check that all .csv have the same number of columns

In [39]:
# Get list of csv with classification feature of individual tiles
import glob
list_feature_csv = glob.glob(os.path.join(config_parameters['outputfolder_classfeatures'],"features_tile_*.csv"))
print "%s .csv files with classification feature in folder"%len(list_feature_csv)
#print list_feature_csv

# Verify if all .csv files have the same number of columns
CheckNumColCsv(list_feature_csv)

6350 .csv files with classification feature in folder
Expected number of columns to be found in all files is 43 (correspond to tile '6023')
All the files have the same number of columns


### Fix .csv that miss one colomn with "rf_class_prob"

This part of the script should be use with care and only for specific problems well identified. 

In [51]:
# Define the header content of the first file as reference. PLEASE CHECK IF IT IS WELL the case. If not, choose another .csv in the list which can be use as reference
fin = open(list_feature_csv[0],"r")
reader = csv.reader(fin)

# Save list with name of columns in the reference file
ref_columns = reader.next()

from os import remove
from shutil import move

for _file in list_feature_csv:
    #Decompose path into path and filename
    path, filename = os.path.split(_file)
    fin = open(_file,"r")
    reader = csv.reader(fin)
    file_header = reader.next()  #Save header of current file
    # Check if file header have the same header than reference file - Continue the loop on the next item if header is the same as reference one 
    if file_header == ref_columns:
        continue
    #Find extra column(s) and its index
    extra = [(column,index) for index,column in enumerate(file_header) if column not in ref_columns] 
    if extra:
        extra_column,extra_index = zip(*extra)
    #Find missing column(s) and its index
    missing = [(column,index) for index,column in enumerate(ref_columns) if column not in file_header] 
    if missing:
        missing_column,missing_index = zip(*missing)
    #Print
    if extra:
        print "The file %s have extra column(s) '%s' and will be fixed"%(filename,"','".join(extra_column))
    if missing:
        print "The file %s miss column(s) '%s' and will be fixed"%(filename,"','".join(missing_column))
    else:
        print "ERROR: Files don't have to same header but neither extra of missing column(s) were find. Please check!"
    #Create a new outputfile (tmp) and csv writer
    temp_path = os.path.join(path,"%s.tmp"%filename)
    fou = open(temp_path, 'w')
    writer = csv.writer(fou)    
    #Create content list for the new file
    content = []
    content.append(ref_columns) #Add header of the reference file
    for row in reader:
        new_row = row
        if extra:  #Remove values at indexes where there is an extra column  
            for index in extra_index:
                del new_row[index]       
        if missing:  #Insert 0 value at indexes where there is a missing column
            for index in missing_index:
                new_row.insert(index,0) 
        content.append(new_row)#Append the new row to the content list
    #Write new content in the new csv
    writer.writerows(content)
    #Rename the temporary file to overwrite original file
    remove(_file)
    move(temp_path, _file)

The file features_tile_6232.csv miss column(s) 'rf_prob_class_942' and will be fixed
The file features_tile_5200.csv miss column(s) 'rf_prob_class_942' and will be fixed


## Get file with training sample (features + label)

In [40]:
# Define path to training file
trainingfile = os.path.join(config_parameters['outputfolder_training_sample'],"training_samples.csv")

In [41]:
import os,sys
import copy
overwrite=True
outputcsv = trainingfile
# Check
if os.path.isfile(outputcsv) and overwrite==False:
    print "File '"+str(outputcsv)+"' aleady exists and overwrite option is not enabled."
    sys.exit()
else:
    if os.path.isfile(outputcsv) and overwrite==True:
        open(outputcsv,"r").close
        os.remove(outputcsv)

# Initialization
available_csv = [os.path.split(a)[1].split(".csv")[0].split("_")[-1] for a in list_feature_csv]
remaining_dic = copy.deepcopy(training_dic)
select_dic = {}
found_counter=0  #To count how many items will be find
fout = open(outputcsv,"a")  #Output csv
writer = csv.writer(fout)   #Writer for output
first_file = True
# Start looking in list of .csv files
for tileid in training_dic:
    print "tileid is %s"%tileid
    if tileid in available_csv:
        featurecsv = os.path.join(config_parameters['outputfolder_classfeatures'],"features_tile_%s.csv"%tileid)
        print featurecsv
        fin = open(featurecsv,"r")
        reader = csv.reader(fin)
        if first_file:  # If first .csv in the list
            header = reader.next()
            header.append("training_label")  # Add "training_label" as column name in last position of the header
            writer.writerow(header)
            first_file = False
        for line in reader:
            if len(remaining_dic[tileid])==0: #If no remaining segid to look for in this tile
                break
            current_segid=line[0]
            if current_segid in training_dic[tileid]:
                print "current_segid is %s"%current_segid
                label = training_dic[tileid][current_segid]
                print label
                newrow = line
                newrow.append(label)  # Add the label in last position of the new row
                writer.writerow(newrow)
                found_counter+=1
                del remaining_dic[tileid][current_segid]
                if label not in select_dic:
                    select_dic[label] = []
                select_dic[label].append(1)
fout.close()
count_remain = len([remaining_dic[subdic][key] for subdic in remaining_dic for key in remaining_dic[subdic]])
if found_counter==0:
    print "ERROR: Nothing found"
    os.remove(outputcsv)
    sys.exit()
if count_remain>0:  #If the research goes through all the .csv files in the list but still remain unfind items
    print "WARNING: %s item(s) not found."%count_remain
print "Process finished"

tileid is 4789
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4789.csv
current_segid is 9336
11
tileid is 4787
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4787.csv
current_segid is 2554
42
current_segid is 35398
5
current_segid is 40835
5
current_segid is 1906
3
current_segid is 29174
5
current_segid is 22587
5
current_segid is 5733
5
tileid is 4785
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4785.csv
current_segid is 7861
5
current_segid is 39581
5
current_segid is 42380
41
current_segid is 43490
5
current_segid is 45541
41
current_segid is 43132
6
tileid is 4781
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4781.csv
current_segid is 25945
5
current_segid is 52597
5
current_segid is 33363
2
current_segid is 16163
5
current_segid is 54389
5
current_segid is 49388
11
current_segid is 57229
2
cur

current_segid is 48341
41
current_segid is 57905
3
current_segid is 63974
3
current_segid is 30182
5
current_segid is 44595
42
tileid is 3679
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3679.csv
current_segid is 88398
3
current_segid is 125817
3
current_segid is 758
11
current_segid is 24799
5
current_segid is 107233
3
tileid is 276
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_276.csv
current_segid is 799
5
current_segid is 10566
5
current_segid is 4179
5
current_segid is 8352
5
current_segid is 14591
5
tileid is 277
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_277.csv
current_segid is 115765
5
current_segid is 24816
42
current_segid is 33988
5
current_segid is 27503
5
current_segid is 104455
5
current_segid is 23573
42
current_segid is 8179
42
current_segid is 26718
5
current_segid is 13133
41
current_segid is 114640
5
current_segid is 391

current_segid is 12638
3
current_segid is 46973
12
current_segid is 34399
3
current_segid is 11460
3
current_segid is 14558
3
current_segid is 59316
11
current_segid is 79792
5
current_segid is 11280
5
tileid is 1840
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1840.csv
current_segid is 110597
5
current_segid is 116991
42
current_segid is 184858
12
current_segid is 54383
5
current_segid is 147146
5
current_segid is 182196
11
current_segid is 21935
42
current_segid is 62938
11
current_segid is 23902
42
current_segid is 78544
11
current_segid is 148705
5
tileid is 1847
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1847.csv
current_segid is 134617
12
current_segid is 134844
42
current_segid is 129741
5
current_segid is 108029
11
current_segid is 40854
42
current_segid is 97584
42
current_segid is 2917
11
current_segid is 48167
11
tileid is 1846
/export/miro/cbassine/fusion/RF_fusion/Results_RW

current_segid is 26660
2
current_segid is 2621
5
tileid is 1177
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1177.csv
current_segid is 135574
12
current_segid is 26584
42
current_segid is 71166
11
current_segid is 217516
12
current_segid is 73893
11
current_segid is 178333
11
tileid is 612
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_612.csv
current_segid is 111281
5
tileid is 1179
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1179.csv
current_segid is 23762
2
current_segid is 24327
11
current_segid is 84232
5
current_segid is 33140
3
tileid is 1178
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1178.csv
current_segid is 43058
5
current_segid is 55094
3
current_segid is 24946
3
current_segid is 148902
5
current_segid is 98168
5
current_segid is 58407
5
current_segid is 111778
5
current_segid is 1

current_segid is 88656
42
current_segid is 121358
11
current_segid is 130647
42
current_segid is 216870
12
current_segid is 360821
12
current_segid is 275825
41
current_segid is 101443
5
current_segid is 246979
5
current_segid is 128936
12
current_segid is 142346
11
current_segid is 312615
11
current_segid is 83595
11
current_segid is 409971
11
tileid is 5267
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_5267.csv
current_segid is 7164
5
current_segid is 1241
5
tileid is 3830
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3830.csv
current_segid is 21381
42
current_segid is 13113
3
current_segid is 51066
42
current_segid is 19388
11
current_segid is 19027
3
current_segid is 45205
41
tileid is 3831
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3831.csv
current_segid is 49250
5
current_segid is 48943
5
current_segid is 11504
42
current_segid is 6749

...

current_segid is 24455
3
current_segid is 18807
3
current_segid is 4636
3
tileid is 3722
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3722.csv
current_segid is 144217
3
current_segid is 144337
3
current_segid is 117819
12
tileid is 3723
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3723.csv
current_segid is 64961
5
current_segid is 28570
6
current_segid is 81743
3
current_segid is 74525
5
current_segid is 87628
3
current_segid is 1110
2
current_segid is 62897
42
current_segid is 83500
3
current_segid is 75721
42
current_segid is 75315
3
current_segid is 87275
3
current_segid is 73505
3
current_segid is 57971
12
tileid is 6357
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_6357.csv
current_segid is 13733
5
current_segid is 28998
5
current_segid is 8521
5
current_segid is 853
3
current_segid is 6574
3
current_segid is 23620
11
current_segid is 15

current_segid is 43671
5
current_segid is 119600
5
current_segid is 182640
11
tileid is 648
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_648.csv
current_segid is 16860
3
tileid is 4797
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4797.csv
current_segid is 39113
42
current_segid is 6243
3
current_segid is 9460
42
tileid is 3544
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_3544.csv
current_segid is 44561
5
current_segid is 4799
42
current_segid is 4102
5
current_segid is 15912
42
tileid is 4019
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4019.csv
current_segid is 62486
42
current_segid is 53936
41
current_segid is 11431
42
current_segid is 10793
42
current_segid is 82993
42
current_segid is 104366
5
current_segid is 18755
41
tileid is 4157
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH

current_segid is 32630
5
current_segid is 52852
11
current_segid is 22458
5
current_segid is 27876
2
current_segid is 56578
5
current_segid is 18285
5
tileid is 2398
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_2398.csv
current_segid is 88464
5
current_segid is 63639
5
current_segid is 4810
5
current_segid is 57409
5
current_segid is 13316
5
current_segid is 61402
42
current_segid is 69597
5
current_segid is 8104
5
current_segid is 51876
2
tileid is 2399
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_2399.csv
current_segid is 1261
42
current_segid is 32258
5
current_segid is 7378
42
current_segid is 49468
5
current_segid is 11528
42
current_segid is 11356
42
tileid is 2770
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_2770.csv
current_segid is 7774
42
current_segid is 27819
42
current_segid is 15399
42
current_segid is 27731
42
current_segid is

current_segid is 35249
3
current_segid is 25002
2
tileid is 4748
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4748.csv
current_segid is 28699
42
current_segid is 9760
42
current_segid is 34853
3
current_segid is 54674
3
current_segid is 37787
5
tileid is 1982
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1982.csv
current_segid is 128770
11
current_segid is 153820
42
current_segid is 16764
42
current_segid is 2760
11
current_segid is 57518
5
current_segid is 73083
42
current_segid is 98365
11
current_segid is 32676
5
current_segid is 162351
2
current_segid is 12436
12
tileid is 4064
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4064.csv
current_segid is 47715
42
current_segid is 23784
5
current_segid is 53659
5
current_segid is 43428
5
tileid is 4065
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_4

current_segid is 148617
11
current_segid is 58766
5
current_segid is 59269
42
current_segid is 45412
42
current_segid is 157543
5
current_segid is 213143
3
current_segid is 217568
11
current_segid is 77290
5
tileid is 1207
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1207.csv
current_segid is 211919
11
current_segid is 181582
12
current_segid is 99580
12
current_segid is 227449
3
current_segid is 224808
5
current_segid is 251447
11
current_segid is 56239
5
current_segid is 213404
5
current_segid is 88293
11
current_segid is 134983
12
current_segid is 219900
11
current_segid is 234758
12
current_segid is 85108
42
tileid is 1206
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_1206.csv
current_segid is 51106
11
current_segid is 10305
12
tileid is 608
/export/miro/cbassine/fusion/RF_fusion/Results_RW_noMNH/Classification_features/features_tile_608.csv
current_segid is 52327
42
current_segid is 44

## Restart the 'cat' id from scratch

Restart the 'cat' id from scratch to avoid duplicate (in case two segment from different tiles have the same id)

In [42]:
with open(trainingfile, 'r') as readFile:
    reader = csv.reader(readFile)
    header = reader.next()
    lines = list(reader)

for i, row in enumerate(lines, 1):
    row[0] = i

with open(trainingfile, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(header)
    writer.writerows(lines)

readFile.close()
writeFile.close()

## Count of point in each class

In [43]:
print "Number of samples per class \n\n"
for label in select_dic:
    print "Class '%s'        %s samples"%(label,sum(select_dic[label]))

Number of samples per class 


Class '11'        324 samples
Class '12'        133 samples
Class '42'        672 samples
Class '41'        334 samples
Class '3'        343 samples
Class '2'        112 samples
Class '5'        924 samples
Class '6'        77 samples


# Train random forest model (R)

### Define parameters for the R script

In [44]:
## Set the path to the output of random forest classification
output_accuracy=os.path.join(config_parameters['outputfolder_classifRF'],"accuracy_RF.csv")
output_varimp=os.path.join(config_parameters['outputfolder_classifRF'],"VariablesImportance_RF.pdf")
output_rfmodel=os.path.join(config_parameters['outputfolder_classifRF'],"RF_model.txt")
output_cv=os.path.join(config_parameters['outputfolder_classifRF'],"RF_cv.pdf")

In [45]:
# Column name with label in training file
train_label = "training_label"

In [46]:
# Set number of cores
r_ncores = 20

### Create R script for the training

In [47]:
# Create and open new temporary R script
r_commands = os.path.join(config_parameters['outputfolder_classifRF'],"Fusion_walous_RF_Train.R").replace("\\", "/")
r_file = open(r_commands, 'w')

In [48]:
# Create a list with all lines to write in the script
r_instuction = []

# Piece of R script for intallation of R package
install_package = "if(!is.element('%s', installed.packages()[,1])){\n"
install_package += "cat('\\n\\nInstalling %s package from CRAN\n')\n"
install_package += "if(!file.exists(Sys.getenv('R_LIBS_USER'))){\n"
install_package += "dir.create(Sys.getenv('R_LIBS_USER'), recursive=TRUE)\n"
install_package += ".libPaths(Sys.getenv('R_LIBS_USER'))}\n"
install_package += "chooseCRANmirror(ind=1)\n"
install_package += "install.packages('%s', dependencies=TRUE)}"
r_instuction.append(install_package % ('doParallel', 'doParallel', 'doParallel')) #Package for parallel processing in R
r_instuction.append(install_package % ('caret', 'caret', 'caret')) #Package for machine learning in R
r_instuction.append(install_package % ('e1071', 'e1071', 'e1071')) 
r_instuction.append(install_package % ('randomForest', 'randomForest', 'randomForest')) #Package for Random Forest
r_instuction.append(install_package % ('pROC', 'pROC', 'pROC')) 

# Piece of R script for loading libraries and dependencies
lib_require = "library(%s)\n"
lib_require += "require(%s)"
r_instuction.append(lib_require % ('parallel', 'parallel')) 
r_instuction.append(lib_require % ('doParallel', 'doParallel')) 
r_instuction.append(lib_require % ('caret', 'caret')) 
r_instuction.append(lib_require % ('randomForest', 'randomForest')) 
r_instuction.append(lib_require % ('e1071', 'e1071')) 
r_instuction.append(lib_require % ('pROC', 'pROC')) 

# Set number of cores to use
r_instuction.append("# Set number of cores to use")
r_instuction.append("usedcores=%s"%r_ncores)
r_instuction.append("usedcores=detectCores()-2")
r_instuction.append("registerDoParallel(usedcores)")

# Import training
r_instuction.append("# Import training")
r_instuction.append("training <- read.csv('%s', sep=',', header=TRUE, row.names=1)"%trainingfile.replace("\\", "/"))
# Define factor
r_instuction.append("# Define factor")
r_instuction.append("training$%s <- as.factor(training$%s)"%(train_label,train_label))

# Cross-validation setting
r_instuction.append("# Cross-validation setting")
r_instuction.append("MyFolds.cv <- createMultiFolds(training$%s, k=5, times=10)"%train_label)
r_instuction.append("MyControl.cv <- trainControl(method='repeatedCV', index=MyFolds.cv, allowParallel = TRUE)")

# Train Random Forest
r_instuction.append("# Train Random Forest")
r_instuction.append("rfModel <- train(%s~.,training,method='rf', trControl=MyControl.cv,tuneLength=10)"%train_label)
r_instuction.append("resamps.cv <- rfModel$resample")
r_instuction.append("accuracy_means <- mean(resamps.cv$Accuracy)")
r_instuction.append("kappa_means <- mean(resamps.cv$Kappa)")
r_instuction.append("df_means <- data.frame(method='rf',accuracy=accuracy_means, kappa=kappa_means)")
r_instuction.append("write.csv(df_means, '%s', row.names=FALSE, quote=FALSE)"%output_accuracy.replace("\\", "/"))

# Plot variable importance
r_instuction.append("# Plot variable importance")
r_instuction.append("importance <- varImp(rfModel, scale=FALSE)")
#r_instuction.append("pdf('%s', width = 11, height = ncol(features)/6 )"%output_varimp.replace("\\", "/"))
r_instuction.append("pdf('%s', width = 11, height = ncol(training)/6 )"%output_varimp.replace("\\", "/"))
r_instuction.append("print(plot(importance))")
r_instuction.append("dev.off()")

# Show final model
r_instuction.append("# Show final model")
r_instuction.append("tmp<-rfModel$finalModel")
r_instuction.append("sink('%s')"%output_rfmodel.replace("\\", "/"))
r_instuction.append("print(tmp)")
r_instuction.append("sink()")

# Plot cross validation tuning results
r_instuction.append("# Plot cross validation tuning results")
r_instuction.append("pdf('%s')"%output_cv.replace("\\", "/"))
r_instuction.append("print(plot(rfModel))")
r_instuction.append("dev.off()")

# Export trained model in file
r_instuction.append("# Export trained model in file")
r_instuction.append("saveRDS(rfModel, file = '%s')"%config_parameters['rf_trained_model'].replace("\\", "/"))

# Write instructions to the R script file
r_file.write("\n".join(r_instuction))
r_file.close()

### Execute R script

In [49]:
import subprocess

In [50]:
start_trainmodel = start_processing()
print "Running R now. Following output is R output."
try:
    subprocess.check_call(['Rscript', r_commands], stderr=subprocess.STDOUT, )
except subprocess.CalledProcessError:
    print "There was an error in the execution of the R script.\nPlease check the R output."
print "Finished running R."
print_processing_time(start_trainmodel, "Computation (on %s cores) achieved in "%r_ncores)

Running R now. Following output is R output.
Finished running R.


'Computation (on 20 cores) achieved in 4 minutes and 41.3 seconds'

### Display results of R

In [51]:
import pandas as pd 
#### Show mean accuracy results from cross-validation for tuning
## Import .csv file
accuracy=pd.read_csv(output_accuracy, sep=',',header=0)
## Display table
accuracy.head(15)

,method,accuracy,kappa
0,rf,0.794316,0.741501


In [52]:
#### Import classifiers tuning parameters and confusion matrix
## Open file
classifier_runs = open(output_rfmodel, 'r')  
## Read file
print classifier_runs.read()


Call:
 randomForest(x = x, y = y, mtry = param$mtry) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 6

        OOB estimate of  error rate: 20.66%
Confusion matrix:
    2   3   5  6  11 12  41  42 class.error
2  17   2  38  2  35  1   8   9   0.8482143
3   2 285  54  0   0  1   0   1   0.1690962
5   8  34 791  1  18  1  22  49   0.1439394
6   0   0   3 64   3  1   2   4   0.1688312
11  3   0  27  3 265 20   0   6   0.1820988
12  0   0   5  1  36 87   0   4   0.3458647
41  0   0  14  0   0  0 274  46   0.1796407
42  0   1  82  3   2  2  49 533   0.2068452



The following cell display pdf figures. Please first install the **Wand** package using the folowwing command in the terminal : 

``` sudo pip instal Wand ```

In [24]:
## Display graphs
import wand
from wand.image import Image as WImage
img = WImage(filename=output_varimp)
img

ImportError: No module named wand

In [53]:
## Display graphs
import wand
from wand.image import Image as WImage
img = WImage(filename=output_cv)
img

ImportError: No module named wand